# Using FourierFT for sequence classification

In this example, we fine-tune Roberta (base) on a sequence classification task using FourierFT.

## Imports

In [1]:
#  To run this notebook, please run `pip install evaluate` to install additional dependencies not covered by PEFT.
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_model,
    FourierFTConfig,
    PeftType,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed, AutoConfig
from tqdm import tqdm

/home/zgaoat/anaconda3/envs/pr2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [2]:
batch_size = 32
model_name_or_path = "roberta-base"
task = "mrpc"
peft_type = PeftType.FOURIERFT
device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 5  # for better results, increase this number
n_frequency = 1000        # for better results, increase this number
scaling = 150.0
max_length = 512
torch.manual_seed(0)

In [3]:
peft_config = FourierFTConfig(
    task_type="SEQ_CLS", 
    n_frequency=n_frequency,
    target_modules=["query", "value"],
    scaling = scaling,
)
head_lr = 6e-3  # the learning rate for the classification head for NLU tasks
fft_lr = 6e-2   # the learning rate for the parameters other than the classification head (q,v in this case)

## Loading data

In [4]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [5]:
datasets = load_dataset("glue", task)
metric = evaluate.load("glue", task)

In [6]:
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=max_length)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence1", "sentence2"],
)

# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [7]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

## Preparing the FourierFT model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, max_length=None)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 616,130 || all params: 125,263,300 || trainable%: 0.4919


In [9]:
head_param = list(map(id, model.classifier.parameters()))

others_param = filter(lambda p: id(p) not in head_param, model.parameters()) 

optimizer = AdamW([
    {"params": model.classifier.parameters(), "lr": head_lr},
    {"params": others_param, "lr": fft_lr}
],weight_decay=0.)


# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

## Training

In [10]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 13/13 [00:00<00:00, 41.72it/s]


epoch 0: {'accuracy': 0.8161764705882353, 'f1': 0.8709122203098106}


100%|██████████| 13/13 [00:00<00:00, 42.91it/s]


epoch 1: {'accuracy': 0.8480392156862745, 'f1': 0.8966666666666666}


100%|██████████| 13/13 [00:00<00:00, 42.65it/s]


epoch 2: {'accuracy': 0.8676470588235294, 'f1': 0.9075342465753424}


100%|██████████| 13/13 [00:00<00:00, 42.11it/s]


epoch 3: {'accuracy': 0.8504901960784313, 'f1': 0.8988391376451078}


100%|██████████| 13/13 [00:00<00:00, 43.15it/s]

epoch 4: {'accuracy': 0.8725490196078431, 'f1': 0.9103448275862069}


## Share adapters on the 🤗 Hub

In [11]:
account_id = ...  # your Hugging Face Hub account ID

In [12]:
model.push_to_hub(f"{account_id}/roberta-base-mrpc-peft-fourierft")

/home/zgaoat/anaconda3/envs/pr2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/zgaoat/roberta-base-mrpc-peft-fourierft/commit/064eb35cbb7a1073b4d8fafbeccee43a0a4e37c9', commit_message='Upload model', commit_description='', oid='064eb35cbb7a1073b4d8fafbeccee43a0a4e37c9', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [13]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer

In [14]:
peft_model_id = f"{account_id}/roberta-base-mrpc-peft-fourierft"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Load the FourierFT model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id, config=config)

In [16]:
inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

100%|██████████| 13/13 [00:00<00:00, 43.06it/s]

{'accuracy': 0.8725490196078431, 'f1': 0.9103448275862069}
